# The InvestmentFlowsIndia data-set

The R version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-R/InvestmentFlowsIndia.ipynb)

In [1]:
import pandas as pd    
from sqlalchemy import func, and_, or_, text, Integer
from sqlalchemy.orm import aliased
from sqlalchemy.sql.expression import cast
from plutoPy.model import RoutingSession, InvestmentFlowsIndia
from datetime import date, datetime, timedelta

### get daily DII and FII flows for the last 20 days for the cash market

In [2]:
results = (RoutingSession.session.query(InvestmentFlowsIndia.DiiCashMarket.TIME_STAMP, InvestmentFlowsIndia.DiiCashMarket.SECURITY_TYPE,
                                        func.sum(InvestmentFlowsIndia.DiiCashMarket.BUY_VALUE + InvestmentFlowsIndia.FiiCashMarket.BUY_VALUE).label('BUY'), 
                                        func.sum(InvestmentFlowsIndia.DiiCashMarket.SELL_VALUE + InvestmentFlowsIndia.FiiCashMarket.SELL_VALUE).label('SELL'),
                                        func.sum(InvestmentFlowsIndia.DiiCashMarket.BUY_VALUE + InvestmentFlowsIndia.FiiCashMarket.BUY_VALUE
                                         - InvestmentFlowsIndia.DiiCashMarket.SELL_VALUE - InvestmentFlowsIndia.FiiCashMarket.SELL_VALUE).label('NET'))
            .outerjoin(InvestmentFlowsIndia.FiiCashMarket, and_(InvestmentFlowsIndia.FiiCashMarket.TIME_STAMP == InvestmentFlowsIndia.DiiCashMarket.TIME_STAMP,
                                                                InvestmentFlowsIndia.FiiCashMarket.SECURITY_TYPE == InvestmentFlowsIndia.DiiCashMarket.SECURITY_TYPE))
            .group_by(InvestmentFlowsIndia.DiiCashMarket.TIME_STAMP, InvestmentFlowsIndia.DiiCashMarket.SECURITY_TYPE)
            .order_by(InvestmentFlowsIndia.DiiCashMarket.TIME_STAMP.desc())
            .limit(20)
            .all())

for instance in results:
    print(instance)

(datetime.date(2019, 7, 19), 'Debt', 21882.67, 18711.57, 3171.1000000000004)
(datetime.date(2019, 7, 19), 'Equity', 9508.23, 9932.970000000001, -424.74000000000024)
(datetime.date(2019, 7, 18), 'Debt', 20864.02, 18812.370000000003, 2051.6500000000005)
(datetime.date(2019, 7, 18), 'Equity', 8882.42, 8495.720000000001, 386.70000000000084)
(datetime.date(2019, 7, 17), 'Debt', 19445.079999999998, 15524.91, 3920.1699999999983)
(datetime.date(2019, 7, 17), 'Equity', 8250.57, 8205.93, 44.63999999999987)
(datetime.date(2019, 7, 16), 'Debt', 17733.57, 15581.470000000001, 2152.099999999999)
(datetime.date(2019, 7, 16), 'Equity', 8878.960000000001, 7652.8099999999995, 1226.1500000000005)
(datetime.date(2019, 7, 15), 'Debt', 15756.539999999999, 12142.34, 3614.199999999999)
(datetime.date(2019, 7, 15), 'Equity', 9883.15, 9395.17, 487.98)
(datetime.date(2019, 7, 12), 'Debt', 17062.08, 14750.3, 2311.7799999999997)
(datetime.date(2019, 7, 12), 'Equity', 9471.68, 8574.87, 896.8099999999996)
(datetime.d

/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)


### get instruments traced for DIIs in the derivative market   

In [3]:
results = (RoutingSession.session.query(InvestmentFlowsIndia.DiiDerivativesMarket.SECURITY_TYPE,
                                        func.min(InvestmentFlowsIndia.DiiDerivativesMarket.TIME_STAMP).label('start_dt'),
                                        func.max(InvestmentFlowsIndia.DiiDerivativesMarket.TIME_STAMP).label('end_dt'))
            .group_by(InvestmentFlowsIndia.DiiDerivativesMarket.SECURITY_TYPE)
            .order_by(text('start_dt'))
            .all())

for instance in results:
    print(instance)

('Index Options', datetime.date(2009, 12, 1), datetime.date(2019, 7, 22))
('Stock Options', datetime.date(2009, 12, 1), datetime.date(2019, 7, 22))
('Interest Rate Futures', datetime.date(2009, 12, 1), datetime.date(2019, 7, 22))
('Stock Futures', datetime.date(2009, 12, 1), datetime.date(2019, 7, 22))
('Index Futures', datetime.date(2009, 12, 1), datetime.date(2019, 7, 22))
('STOCK_OPTIONS', datetime.date(2017, 4, 3), datetime.date(2017, 11, 8))
('STOCK_FUTURES', datetime.date(2017, 4, 3), datetime.date(2017, 11, 8))
('INTEREST_RATE_FUTURES', datetime.date(2017, 4, 3), datetime.date(2017, 11, 8))
('INDEX_FUTURES', datetime.date(2017, 4, 3), datetime.date(2017, 11, 8))
('INDEX_OPTIONS', datetime.date(2017, 4, 3), datetime.date(2017, 11, 8))


In [4]:
### get instruments traced for FIIs in the derivative market   

In [5]:
results = (RoutingSession.session.query(InvestmentFlowsIndia.FiiDerivativesMarket.SECURITY_TYPE,
                                        func.min(InvestmentFlowsIndia.FiiDerivativesMarket.TIME_STAMP).label('start_dt'),
                                        func.max(InvestmentFlowsIndia.FiiDerivativesMarket.TIME_STAMP).label('end_dt'))
            .group_by(InvestmentFlowsIndia.FiiDerivativesMarket.SECURITY_TYPE)
            .order_by(text('start_dt'))
            .all())

for instance in results:
    print(instance)

('Index Options', datetime.date(2009, 12, 1), datetime.date(2014, 5, 30))
('Stock Options', datetime.date(2009, 12, 1), datetime.date(2014, 5, 30))
('Interest Rate Futures', datetime.date(2009, 12, 1), datetime.date(2014, 5, 29))
('Stock Futures', datetime.date(2009, 12, 1), datetime.date(2014, 5, 30))
('Index Futures', datetime.date(2009, 12, 1), datetime.date(2014, 5, 30))
('STOCK_OPTIONS', datetime.date(2014, 6, 2), datetime.date(2019, 7, 22))
('STOCK_FUTURES', datetime.date(2014, 6, 2), datetime.date(2019, 7, 22))
('INTEREST_RATE_FUTURES', datetime.date(2014, 6, 2), datetime.date(2019, 7, 22))
('INDEX_FUTURES', datetime.date(2014, 6, 2), datetime.date(2019, 7, 22))
('INDEX_OPTIONS', datetime.date(2014, 6, 2), datetime.date(2019, 7, 22))


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)